In [2]:
import sys
import os
sys.path.append(os.path.abspath('./../..'))
from cell_extractor.CellDetectorBase import CellDetectorBase
from pipeline.lib.annotation_layer import Cell,random_string
import numpy as np
from random import sample
from pipeline.lib.UrlGenerator import UrlGenerator
from cell_extractor.BorderFinder import BorderFinder
def numpy_to_json(data,color_hex = None,category = 'Round3_Sure'):
    cells = []
    for coord in data:
        cell = Cell(np.array(coord,dtype=float),random_string())
        cell.category = category
        cell.coord[-1]=cell.coord[-1]+0.5
        if hasattr(cell,'description'):
            del cell.description
        cell_json = cell.to_json()
        if color_hex is not None:
            cell_json["props"] = [color_hex]
        cells.append(cell_json)
    return cells


In [3]:

animal = 'DK52'
finder = BorderFinder(animal)
base = CellDetectorBase(animal,round = 2)
detections = base.load_detections()
sure = detections[detections.predictions==2]
unsure = detections[detections.predictions==0]

border_cell,sure = finder.find_border_cells(sure)
border_cell,unsure = finder.find_border_cells(unsure)

len(sure),len(unsure),len(detections)
sure_data = sure[['col','row','section']].sample(500).sort_values('section').to_numpy().tolist()
unsure_data = unsure[['col','row','section']].sort_values('section').to_numpy().tolist()
sure_cells = numpy_to_json(sure_data,category = 'Round3_Sure')
unsure_cells = numpy_to_json(unsure_data,color_hex='#1d66db',category = 'Round3_Unsure')
urlGen = UrlGenerator()
image_layer = 'precomputed://https://activebrainatlas.ucsd.edu/data/'+animal+'/neuroglancer_data/'
urlGen.add_stack_image(animal,channel=1)
urlGen.add_stack_image(animal,channel=2,color='red')
urlGen.add_stack_image(animal,channel=3,color='green')
urlGen.add_annotation_layer('Sure',annotations = sure_cells)
urlGen.add_annotation_layer('Unsure',annotations = unsure_cells)
urlGen.add_to_database('test QC3',34)

calculating borders for DK52
image loaded
contours found
borders calculated


590